# Medicijn hergebruik voor kanker



## Intro
Het ontwikkelen van nieuwe medicijnen kost veel geld en tijd, terwijl het niet altijd nodig is om die helemaal opnieuw te ontwikkelen. We beschikken namelijk al over een breed scala aan goedgekeurde medicijnen. Bij colorectale kanker is er sprake van verhoogde genetische instabiliteit, veroorzaakt door hoge LINE-1 transposonactiviteit, wat een kenmerk is van de ziekte. Ieder heeft ongeveer 100 actieve, maar zwaar gemethyleerde LINE-1's in het genoom. Deze functioneren door middel van reverse transcriptase (RT), afkomstig van hun retrovirale achtergrond. Hierdoor is het mogelijk retrovirale virusremmers, zoals HIV-remmers, te gebruiken als medicijn om deze activiteit te onderdrukken. Dit wordt al toegepast in sommige kankertherapieën en is een voorbeeld van medicijn hergebruik. Het biedt een kans om te onderzoeken of er wellicht andere medicijnen zijn die potentieel kunnen worden gebruikt in de behandeling van kanker.

# Doel

Het doel van dit project is om te onderzoeken of er al mogelijk bestaande medicatie is die van toepassing kan zijn op kanker. Dit gaan we onderzoeken door middel van RNA count data van kanker en niet kanker. Hier worden de differential expressed genes (DEG) van afgeleid. Vervolgens een medicijnen netwerk op te zetten op basis van deze DEG. Dit zal leiden tot een netwerk model van potentiele medicijnen.

Als een proof of concept gaan we inzoomen op 1 kankersoort. Dit wordt borstkanker.

# Project Imports

In [1]:
import pandas as pd, numpy as np, matplotlib, seaborn
import matplotlib.pyplot as plt
import networkx as nx
from scipy import stats
from statsmodels.stats.multitest import multipletests


# Exploratieve Data Analysis

## DEG zoeken
Starten met het inladen van de data die uitgepakt staat in `data/`. De data komt van [https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE62944](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE62944).

In [2]:
# Het normal.txt-bestand inladen en het tonen van de eerste paar regels
normal_data = pd.read_csv("data/GSM1697009_06_01_15_TCGA_24.normal_Rsubread_FeatureCounts.txt", sep="\t", index_col=0)
print("Normal data")
print(normal_data.head())

# Het cancer.txt-bestand inladen en het tonen van de eerste paar regels
cancer_data = pd.read_csv("data/GSM1536837_06_01_15_TCGA_24.tumor_Rsubread_FeatureCounts.txt", sep="\t", index_col=0)
print("\n Cancer data")
print(cancer_data.head())


Normal data
             TCGA-06-0675-11A-32R-A36H-07  TCGA-06-0678-11A-32R-A36H-07  \
1/2-SBSRNA4                            48                            25   
A1BG                                  206                           112   
A1BG-AS1                               87                            44   
A1CF                                    8                             2   
A2LD1                                 138                            99   

             TCGA-06-0680-11A-32R-A36H-07  TCGA-06-0681-11A-41R-A36H-07  \
1/2-SBSRNA4                            36                            28   
A1BG                                  223                           361   
A1BG-AS1                              122                           120   
A1CF                                    5                             1   
A2LD1                                 127                           110   

             TCGA-06-AABW-11A-31R-A36H-07  TCGA-22-4593-11A-01R-1820-07  \
1/2-SBSRNA4

Een functie om labels uit het bestand te halen.

In [3]:
# chat-gpt
def process_cancer_data(file_path):
    """
        Processes a file with labels and cancer types and returns a dictionary.

        Args:
            file_path (str): Path to the file.

        Returns:
            dict: A dictionary with cancer types as keys and lists of labels as values.
    """
    cancer_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            # Splits de lijn in label en kanker type
            label, cancer_type = line.strip().split('\t')

            # Voeg toe aan de dictionary
            if cancer_type not in cancer_dict:
                cancer_dict[cancer_type] = []
            cancer_dict[cancer_type].append(label)

    return cancer_dict

Functie toepassen.

In [4]:
# Using function
cancer_key = "BRCA"
normal_data_labels = process_cancer_data("data/GSE62944_06_01_15_TCGA_24_Normal_CancerType_Samples.txt")
print(normal_data_labels[cancer_key])

['TCGA-BH-A0DV-11A-22R-A12P-07', 'TCGA-BH-A0BC-11A-22R-A089-07', 'TCGA-BH-A0BM-11A-12R-A089-07', 'TCGA-AC-A2FB-11A-13R-A17B-07', 'TCGA-A7-A13E-11A-61R-A12P-07', 'TCGA-BH-A0BA-11A-22R-A19E-07', 'TCGA-BH-A1FC-11A-32R-A13Q-07', 'TCGA-E9-A1R7-11A-42R-A14M-07', 'TCGA-BH-A0BQ-11A-33R-A115-07', 'TCGA-AC-A2FF-11A-13R-A17B-07', 'TCGA-BH-A208-11A-51R-A157-07', 'TCGA-BH-A0DT-11A-12R-A12D-07', 'TCGA-BH-A0HA-11A-31R-A12P-07', 'TCGA-BH-A1FD-11B-21R-A13Q-07', 'TCGA-BH-A18L-11A-42R-A12D-07', 'TCGA-A7-A13G-11A-51R-A13Q-07', 'TCGA-AC-A2FM-11B-32R-A19W-07', 'TCGA-BH-A0BS-11A-11R-A12P-07', 'TCGA-BH-A18Q-11A-34R-A12D-07', 'TCGA-BH-A0AZ-11A-22R-A12P-07', 'TCGA-BH-A0AU-11A-11R-A12P-07', 'TCGA-BH-A1F2-11A-32R-A13Q-07', 'TCGA-BH-A0BW-11A-12R-A115-07', 'TCGA-A7-A13F-11A-42R-A12P-07', 'TCGA-E9-A1NF-11A-73R-A14D-07', 'TCGA-BH-A1FU-11A-23R-A14D-07', 'TCGA-BH-A18J-11A-31R-A12D-07', 'TCGA-BH-A0E0-11A-13R-A089-07', 'TCGA-E9-A1NA-11A-33R-A144-07', 'TCGA-A7-A0D9-11A-53R-A089-07', 'TCGA-E2-A158-11A-22R-A12D-07', 'TCGA-A

In [5]:
cancer_data_labels = process_cancer_data("data/GSE62944_01_27_15_TCGA_20_CancerType_Samples.txt")
print(cancer_data_labels[cancer_key])

['TCGA-A7-A425-01A-11R-A24H-07', 'TCGA-BH-A18G-01A-11R-A12D-07', 'TCGA-BH-A0BD-01A-11R-A034-07', 'TCGA-B6-A0IB-01A-11R-A034-07', 'TCGA-B6-A1KN-01A-11R-A13Q-07', 'TCGA-A2-A0YJ-01A-11R-A109-07', 'TCGA-A2-A0YL-01A-21R-A109-07', 'TCGA-A2-A0YI-01A-31R-A10J-07', 'TCGA-AR-A0U0-01A-11R-A109-07', 'TCGA-AR-A0TT-01A-31R-A084-07', 'TCGA-AQ-A54N-01A-11R-A266-07', 'TCGA-AN-A0XV-01A-11R-A109-07', 'TCGA-AO-A03O-01A-11R-A00Z-07', 'TCGA-AO-A128-01A-11R-A10J-07', 'TCGA-AN-A046-01A-21R-A034-07', 'TCGA-AN-A0FL-01A-11R-A034-07', 'TCGA-E9-A1NE-01A-21R-A14M-07', 'TCGA-E2-A3DX-01A-21R-A213-07', 'TCGA-E2-A15D-01A-11R-A115-07', 'TCGA-D8-A1XZ-01A-11R-A14M-07', 'TCGA-D8-A27E-01A-11R-A16F-07', 'TCGA-D8-A1XS-01A-11R-A14M-07', 'TCGA-A8-A0AB-01A-11R-A034-07', 'TCGA-A8-A0A7-01A-11R-A00Z-07', 'TCGA-A8-A09X-01A-11R-A00Z-07', 'TCGA-A7-A26H-01A-11R-A169-07', 'TCGA-OL-A5DA-01A-11R-A27Q-07', 'TCGA-MS-A51U-01A-31R-A266-07', 'TCGA-BH-A204-01A-11R-A157-07', 'TCGA-BH-A1F0-01A-11R-A137-07', 'TCGA-BH-A0W4-01A-11R-A109-07', 'TCGA-B

Nu Gaan we de datasets subsetten op kankertype. We kiezen voor de `BCRA`, dat borstkanker representeerd

In [6]:
# Making BRCA labels
normal_BRCA_labels = normal_data_labels[cancer_key]
cancer_BRCA_labels = cancer_data_labels[cancer_key]
print(f"Normal BRCA label: {len(normal_BRCA_labels)} | Cancer BRCA label: {len(cancer_BRCA_labels)}")

# Subsetting data to BRCA key
print(f"Normal dataset length: {len(normal_data.columns)} | Cancer dataset length: {len(cancer_data.columns)}")
normal_data_subset = normal_data[normal_BRCA_labels]
cancer_data_subset = cancer_data[cancer_BRCA_labels]
print(f"After subsetting\nNormal dataset length: {len(normal_data_subset.columns)} | Cancer dataset length: {len(cancer_data_subset.columns)}")


Normal BRCA label: 113 | Cancer BRCA label: 1082
Normal dataset length: 741 | Cancer dataset length: 9264
After subsetting
Normal dataset length: 113 | Cancer dataset length: 1082


De counts moeten nog genormaliseerd worden. Dit wordt met een log2 transformatie toegepast op de dataset. Een tpm is ook nog een overwogen optie, alleen voldoet voor dit korte onderzoek de log2.

In [7]:
def log_transform(expression_data):
    """
    Past een log2(x + 1) transformatie toe op een expressiematrix.

    Args:
        expression_data (pd.DataFrame): Raw counts data waar rijen genen zijn en kolommen samples.

    Returns:
        pd.DataFrame: Log-getransformeerde expressiegegevens.
    """
    return np.log2(expression_data + 1)

Nu het toepassen van de normalisering

In [8]:
normal_data_log = log_transform(normal_data_subset)
cancer_data_log = log_transform(cancer_data_subset)

Nu het zoeken naar de DEG. Hiervoor gebruiken we meervoudige t-test.

In [9]:
# Maak een lege lijst om de resultaten op te slaan
deg_results_list = []

# Loop over de genen in de subset
for gene in normal_data_subset.index:
    # Haal de genexpressiewaarden voor normaal en kanker
    normal_values = normal_data_log.loc[gene].values
    cancer_values = cancer_data_log.loc[gene].values

    # Zorg ervoor dat beide groepen gegevens bevatten
    if len(normal_values) == 0 or len(cancer_values) == 0:
        continue  # Sla over als er geen waarden zijn

    # Voer de t-test uit
    t_stat, p_value = stats.ttest_ind(normal_values, cancer_values, nan_policy='omit')  # nan_policy='omit' om NaN te negeren

    # Controleer of p_value geldig is (geen NaN)
    if np.isnan(p_value):
        continue  # Sla genen met een NaN p-waarde over

    # Bereken de log2 fold change
    log2_fold_change = (cancer_values.mean() + 1e-9) / (normal_values.mean() + 1e-9)
    log2_fold_change = np.log2(log2_fold_change)

    # Voeg de resultaten toe aan de lijst
    deg_results_list.append({'gene': gene, 'p_value': p_value, 'log2_fold_change': log2_fold_change})

# Zet de lijst om naar een DataFrame
deg_results = pd.DataFrame(deg_results_list)

# Controleer of deg_results leeg is
print(f"Number of results: {len(deg_results)}")

# Corrigeer voor meervoudige tests (bijvoorbeeld met de Benjamini-Hochberg methode)
deg_results['adjusted_p_value'] = multipletests(deg_results['p_value'], method='fdr_bh')[1]

# Filter voor significante genen (bijvoorbeeld p-value < 0.05)
significant_genes = deg_results[(deg_results['adjusted_p_value'] < 0.05) & (deg_results['log2_fold_change'].abs() > 1.5)]

# Toon de significante genen
print(significant_genes)


Number of results: 22828
                gene       p_value  log2_fold_change  adjusted_p_value
16           AADACL2  5.946168e-57         -2.053206      1.393625e-55
17           AADACL3  5.175597e-47         -1.981595      8.751744e-46
147            ACCN5  2.315207e-08          2.064753      6.013375e-08
198           ACSM2A  1.131945e-77         -1.854185      5.617400e-76
199           ACSM2B  2.678361e-69         -1.903352      9.814064e-68
...              ...           ...               ...               ...
21744        WBSCR28  1.204012e-52          1.992041      2.485099e-51
21894         WT1-AS  2.259882e-23          1.659053      1.407217e-22
21909          XAGE3  6.182491e-49         -2.147741      1.108672e-47
22612  ZNF670-ZNF695  9.198208e-13          2.282103      3.185809e-12
22782            ZP4  4.736589e-08          2.169195      1.203549e-07

[397 rows x 4 columns]


In [10]:
print(len(significant_genes))
print(significant_genes.head())

397
        gene       p_value  log2_fold_change  adjusted_p_value
16   AADACL2  5.946168e-57         -2.053206      1.393625e-55
17   AADACL3  5.175597e-47         -1.981595      8.751744e-46
147    ACCN5  2.315207e-08          2.064753      6.013375e-08
198   ACSM2A  1.131945e-77         -1.854185      5.617400e-76
199   ACSM2B  2.678361e-69         -1.903352      9.814064e-68


Top we hebben dus DEGs. Nu moeten we onderzoeken welke medicijnen hierbij gepaard gaan.

## Matchen van Medicijnen

Nu eenmaal in bezit van DEGs is de volgende taak, onderzoeken welke bekende medicijnen met deze genen betrokken zijn. Hiervoor zoeken we een dataset die medicijnen verbind aan genen. Een data zoals die van NIH LINCS program. De data staat op [https://maayanlab.cloud/sigcom-lincs/#/Download](https://maayanlab.cloud/sigcom-lincs/#/Download). Deze dataset heeft de vorm 4269x35239, ik ga er vanuit dat al onze deg zich hierin bevinden. Nu eerst de data inlezen.

In [41]:
drug_data = normal_data = pd.read_csv("data/human_geo_sigs.tsv", sep="\t", index_col=0)
print(drug_data.head())

                                                        A1BG      A1CF  \
Identification of differentially expressed circ... -2.198815 -0.297826   
CRISPR Inversion of CTCF Sites Alters Genome To...  0.517292  0.000000   
mRNA differential expression in LNCaP cells exp...  0.208042  0.017953   
EHMT1 and EHMT2 inhibition induce fetal hemoglo...  0.167852  0.000000   
Prolyl hydroxylation regulates protein degradat...  0.340173 -0.002414   

                                                         A2M     A2ML1  \
Identification of differentially expressed circ... -2.457639 -1.564745   
CRISPR Inversion of CTCF Sites Alters Genome To...  0.000000  0.000000   
mRNA differential expression in LNCaP cells exp...  1.650377 -0.820223   
EHMT1 and EHMT2 inhibition induce fetal hemoglo...  0.000000  0.018627   
Prolyl hydroxylation regulates protein degradat... -0.109122 -0.060994   

                                                       A2MP1  A3GALT2  \
Identification of differentially expr

Dit ziet er veel belovend uit. De shape komt op 1 na overeen met wat de beschrijving is. Nu bekijken of al onze genen ook aanwezig zijn. in onze dataset.

In [42]:
# Controleren welke genen uit significant_genes niet in drug_data kolommen zitten
present_genes = significant_genes.loc[:, "gene"][significant_genes.loc[:, "gene"].isin(drug_data.columns)]
print("Significant genes: ", len(significant_genes))
print("Present genes:", len(present_genes))
print("Missing genes: ", len(significant_genes) - len(present_genes))

Significant genes:  397
Present genes: 244
Missing genes:  153


Helaas missen er een aantal genen, maar gelukkig zijn er veel wel aanwezig. We gaan met de aanwezige genen verder! Dus gaan we een subset maken en hier mee verder.

In [43]:
drug_data_subset = drug_data[present_genes]
significant_genes_subset = significant_genes.set_index("gene").loc[present_genes]
print(f"Drug_data genes: {len(drug_data_subset.count())} | Significant genes: {len(significant_genes_subset)}")

Drug_data genes: 244 | Significant genes: 244


Dat is goed gegaan. Nu we de data concreet hebben gaan we een correlatie analyze uitoefenen. Dit doen we om te onderzoeken welke medicijnen het sterkst met deze genen gecorreleerd worden.

In [44]:
print(deg_results.head())
# Zorg ervoor dat drug_data_subset en significant_genes_subset op dezelfde samples zijn gebaseerd
common_samples = drug_data_subset.index.intersection(significant_genes_subset.columns)
drug_data_subset = drug_data_subset.loc[common_samples]
significant_genes_subset = significant_genes_subset[common_samples]

# Bereken de Pearson-correlaties
correlation_matrix = pd.DataFrame(index=drug_data_subset.columns, columns=significant_genes_subset.index)

for drug in drug_data_subset.columns:
    for gene in significant_genes_subset.index:
        corr, _ = stats.pearsonr(drug_data_subset[drug], significant_genes_subset.loc[gene])
        correlation_matrix.loc[drug, gene] = corr

# Het bekijken van de top 10 meest gecorreleerde medicijnen
top_drugs = correlation_matrix.max(axis=1).sort_values(ascending=False).head(10)
print(top_drugs)


          gene       p_value  log2_fold_change  adjusted_p_value
0  1/2-SBSRNA4  8.087410e-01         -0.005301      8.391028e-01
1         A1BG  2.488353e-26          0.239430      1.772796e-25
2     A1BG-AS1  3.295560e-09          0.128450      9.059615e-09
3         A1CF  2.941600e-01         -0.090663      3.506571e-01
4        A2LD1  6.497187e-01          0.006764      6.978018e-01


ValueError: `x` and `y` must have length at least 2.

Dit resultaat duit op een misverstand. Mijn resultaat van wat ik verwachte wijkt af, ik heb namelijk relevante studies op basis van mijn deg. Niet relevante medicijnen. Dit komt door het niet goed onderzoeken van mijn data en wat het precies bevat. Conclusie: Het verbinden van expressiedata en data afkomstig van relevante studies is mogelijk, dus dan zal het ook mogelijk moeten zijn om dit met medicijn-data te doen. Op zichzelf toont dit aan dat het gebruiken van expressie data vele kanten en mogelijkheden biedt, en ook invalshoeken die je soms niet verwacht of al onderzocht zijn.

Voor nu weer een stap terug, het zoeken van Medicijn data.

Er is een handmatig geannoteerde dataset die genen aan medicijnen verbindt. Deze bevat 875 verschillende middelen met elk relevante genen. Met mijn lijst van DEGs kan ik het hiermee in een netwerk leggen. De data is afkomstig uit [https://maayanlab.cloud/CREEDS/](https://maayanlab.cloud/CREEDS/), de v1.0	Manual single drug perturbations. Eerst kijken of het echt over middelen gaat, aangezien dit eerder wel beter gecontroleerd kon worden.

In [17]:
drug_file = "data/single_drug_perturbations-v1.0.gmt"

with open(drug_file, 'r') as file:
    drug_file = {}
    for line in file:
        parts = line.strip().split('\t')
        drug_file[parts[0]] = parts[2:]

# Bekijk een voorbeeld
for gene_set, genes in drug_file.items():
    print(f"Genenset: {gene_set}, Aantal genen: {len(genes)}")

# Bekijk laatste set genen
print(genes)

Genenset: Fluorouracil-up, Aantal genen: 314
Genenset: Fluorouracil-dn, Aantal genen: 283
Genenset: Resveratrol-up, Aantal genen: 359
Genenset: Resveratrol-dn, Aantal genen: 241
Genenset: Citalopram-up, Aantal genen: 341
Genenset: Citalopram-dn, Aantal genen: 250
Genenset: Ethanol-up, Aantal genen: 331
Genenset: Ethanol-dn, Aantal genen: 240
Genenset: Sodium arsenite-up, Aantal genen: 309
Genenset: Sodium arsenite-dn, Aantal genen: 284
Genenset: Vitamin c-up, Aantal genen: 250
Genenset: Vitamin c-dn, Aantal genen: 322
Genenset: Perfluorooctanoic acid-up, Aantal genen: 218
Genenset: Perfluorooctanoic acid-dn, Aantal genen: 373
Genenset: Formaldehyde-up, Aantal genen: 271
Genenset: Formaldehyde-dn, Aantal genen: 329
Genenset: Tretinoin-up, Aantal genen: 359
Genenset: Tretinoin-dn, Aantal genen: 241
Genenset: Vitamin e-up, Aantal genen: 258
Genenset: Vitamin e-dn, Aantal genen: 340
Genenset: Gefitinib-up, Aantal genen: 117
Genenset: Gefitinib-dn, Aantal genen: 483
Genenset: Methylpredniso

Top er zijn genen aanwezig. Maar zijn het ook de genen die ik heb? Dat gaan we bekijken.

In [27]:
total_hits = 0
relevant_gene_sets = []
for gene_set, genes in drug_file.items():
   present_genes_count =  len([gene for gene in genes if gene in significant_genes_subset.index])
   if present_genes_count > 0:
       relevant_gene_sets.append(gene_set)

   print(f"Gene set: {gene_set}, Present gene count: {present_genes_count}")

   total_hits += present_genes_count

print(f"Total hits: {total_hits}")
print(f"Relevant drugs count:{len(relevant_gene_sets)}")


Gene set: Fluorouracil-up, Present gene count: 0
Gene set: Fluorouracil-dn, Present gene count: 0
Gene set: Resveratrol-up, Present gene count: 0
Gene set: Resveratrol-dn, Present gene count: 0
Gene set: Citalopram-up, Present gene count: 0
Gene set: Citalopram-dn, Present gene count: 0
Gene set: Ethanol-up, Present gene count: 1
Gene set: Ethanol-dn, Present gene count: 2
Gene set: Sodium arsenite-up, Present gene count: 0
Gene set: Sodium arsenite-dn, Present gene count: 0
Gene set: Vitamin c-up, Present gene count: 0
Gene set: Vitamin c-dn, Present gene count: 0
Gene set: Perfluorooctanoic acid-up, Present gene count: 0
Gene set: Perfluorooctanoic acid-dn, Present gene count: 0
Gene set: Formaldehyde-up, Present gene count: 0
Gene set: Formaldehyde-dn, Present gene count: 0
Gene set: Tretinoin-up, Present gene count: 0
Gene set: Tretinoin-dn, Present gene count: 0
Gene set: Vitamin e-up, Present gene count: 0
Gene set: Vitamin e-dn, Present gene count: 0
Gene set: Gefitinib-up, Pres

Er zijn genen aanwezig. Nu kijken of we hier een netwerk van kunnen maken.

In [53]:
# Maak een lege graaf
G = nx.Graph()

# Voeg genen en medicijnen toe als knopen
for drug, genes in drug_file.items():
    for gene in genes:
        if gene in significant_genes_subset.index:
            G.add_edge(drug, gene)

# Pas node-groottes en labels aan
for node in G.nodes:
    G.nodes[node]['font'] = {'size': 16}
    print(f"Node: {node}")
    if node in drug_file:  # Node is een medicijn
        G.nodes[node]['size'] = 20
        G.nodes[node]['label'] = f"Drug: {node}"
        G.nodes[node]['color'] = 'blue'
    elif node in significant_genes_subset.index:  # Node is een gen
        G.nodes[node]['size'] = 15
        G.nodes[node]['label'] = f"Gene: {node}"
        G.nodes[node]['color'] = 'green'


# Visualiseren
from pyvis.network import Network

# Pyvis Network object

net = Network(notebook=True)  # Als je in Jupyter werkt
net.from_nx(G)  # Converteer NetworkX-graaf naar Pyvis

net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.3,
      "springLength": 95,
      "springConstant": 0.04,
      "damping": 0.09
    },
    "minVelocity": 0.75
  }
}
""")


net.show("network.html")

Node: Ethanol-up
Node: GPR75-ASB3
Node: Ethanol-dn
Node: PMP2
Node: HHATL
Node: Gefitinib-up
Node: FTHL17
Node: Gefitinib-dn
Node: HIST1H4L
Node: HIST1H4B
Node: HIST1H3C
Node: H3F3A
Node: HOXB13
Node: HIST1H3B
Node: HIST1H2AJ
Node: 4-hydroxytamoxifen-dn
Node: BPIFA4P
Node: Mycophenolic acid-dn
Node: HBG2
Node: Abiraterone-up
Node: INS-IGF2
Node: Abiraterone-dn
Node: HIST1H2BM
Node: HIST1H3I
Node: HIST1H1T
Node: HIST1H4D
Node: SPRR2F
Node: Cisplatin-dn
Node: HIST1H2AB
Node: Metoprolol-up
Node: CA4
Node: Metoprolol-dn
Node: RNF186
Node: CRYBA2
Node: ARL14
Node: SGCZ
Node: Methotrexate-up
Node: Decitabine-up
Node: MAGEA3
Node: MAGEA6
Node: Estradiol-dn
Node: Etanercept-up
Node: CARD18
Node: SPRR2B
Node: TMEFF2
Node: Lipopolysaccharide-up
Node: CCL7
Node: Sorafenib-up
Node: CST1
Node: Sorafenib-dn
Node: Lapatinib-dn
Node: Bisphenol a-dn
Node: Lysophosphatidic acid-up
Node: Lysophosphatidic acid-dn
Node: CST2
Node: CST4
Node: Adenosine triphosphate-up
Node: IL19
Node: Torcetrapib-up
Node: N